## Downloading Data from OSM

Open street map data can be received from an API call in the XML format, basically a hierarchical dictionary structure that looks like HTML.

We only care about the [power](https://openinframap.org/#14.43/26.06828/-80.20208) key, especially generators, substations, lines, transformers, and switches. 
Here we map the [Ft. Lauderdale Power Plant](https://openinframap.org/#14.43/26.06828/-80.20208), which consists of three power-plants and a few substations. 

The raw XML data can be downloaded from the [OSM website](https://www.openstreetmap.org/export#map=16/26.0720/-80.2004), some is shown below. 

OSM data is a `node`, `way`, or `relation`. Nodes are the basis for everything and store geospatial data as `lat`, `lon` attributes. Ways are a list of nodes that represent either lines or polygons. Relations are groups of nodes and ways and can represent just about anything, like a power plant. 

In [ ]:
<!-- GENERATION -->
<node id="9816126697" lat="26.0850857" lon="-80.1251280" version="1" timestamp="2022-06-14T01:50:53Z" changeset="122348291" uid="1443767" user="Olyon">
    <tag k="generator:method" v="combustion"/>
    <tag k="generator:output:electricity" v="296 MW"/>
    <tag k="generator:source" v="gas"/>
    <tag k="generator:type" v="gas_turbine"/>
    <tag k="power" v="generator"/>
    <tag k="source" v="U.S. Energy Information Administration - EIA-860 - 2020"/>
    <tag k="start_date" v="2016-04"/>
  </node>
  <!-- POWER PLANT -->
<way id="542155046" version="14" timestamp="2023-02-26T01:59:19Z" changeset="133027405" uid="10661773" user="Local-Mapper">
    <nd ref="4196305262"/>
    <nd ref="5241799909"/>
<!-- many taken out for simplicity -->
    <nd ref="4196305262"/>
    <tag k="addr:city" v="Ft. Lauterdale"/>
    <tag k="addr:housenumber" v="8100"/>
    <tag k="addr:postcode" v="33316"/>
    <tag k="addr:state" v="FL"/>
    <tag k="addr:street" v="Eisenhower Boulevard"/>
    <tag k="landuse" v="industrial"/>
    <tag k="name" v="Port Everglades Power Plant"/>
    <tag k="operator" v="Florida Power &amp; Light"/>
    <tag k="plant:method" v="combustion"/>
    <tag k="plant:output:electricity" v="1352 MW"/>
    <tag k="plant:source" v="gas"/>
    <tag k="plant:type" v="combined_cycle"/>
    <tag k="power" v="plant"/>
    <tag k="ref:US:EIA" v="617"/>
    <tag k="source" v="U.S. Energy Information Administration - EIA-860 - 2020"/>
    <tag k="start_date" v="1960-06"/>
    <tag k="wikidata" v="Q11996062"/>
  </way>
<!-- SUBSTATION -->
 <way id="48829189" visible="true" version="8" changeset="149247712" timestamp="2024-03-28T01:07:31Z" user="Olyon" uid="1443767">
  <nd ref="619471506"/>
  <nd ref="619471479"/>
<!-- many taken out for simplicity -->
  <nd ref="619471506"/>
  <tag k="frequency" v="60"/>
  <tag k="location" v="outdoor"/>
  <tag k="name" v="Lauderdale Plant Substation"/>
  <tag k="operator" v="Florida Power &amp; Light"/>
  <tag k="operator:wikidata" v="Q1430055"/>
  <tag k="power" v="substation"/>
  <tag k="substation" v="transmission"/>
  <tag k="voltage" v="230000;138000;13800"/>
 </way>
<!-- POWER LINE -->
  <way id="48467928" version="6" timestamp="2023-01-28T08:56:56Z" changeset="131799659" uid="13929699" user="Gamer2000">
    <nd ref="3721597479"/>
    <nd ref="615886297"/>
<!-- again, many taken out for simplicity-->
    <nd ref="1340766482"/>
    <tag k="cables" v="3"/>
    <tag k="frequency" v="60"/>
    <tag k="name" v="Laudania Sub FPL to Port Everglades Sub FPL 230KV"/>
    <tag k="operator" v="Florida Power &amp; Light"/>
    <tag k="operator:wikidata" v="Q1430055"/>
    <tag k="power" v="line"/>
    <tag k="voltage" v="230000"/>
  </way>
<!--  WHICH IS A COLLECTION OF POWER TOWERS -->
<node id="615886339" lat="26.0674905" lon="-80.1293144" version="3" timestamp="2015-08-30T15:03:32Z" changeset="33684428" uid="160651" user="dcat">
    <tag k="power" v="tower"/>

Here is a simple script that downloads all power data from a specified area using the OSM API. For simplicity, I'm storing everything in the same `DATA` dictionary, and creating a dictionary of indices for each power type using `POWER_IDXS`. See the function specs for details. 

In [45]:
import requests
import xml.etree.ElementTree as ET
import queue
from collections import defaultdict

# function to convert an integer to a string with the appropriate suffix
Ith = lambda i: str(i) + ("th" if (abs(i) % 100 in (11,12,13)) else ["th","st","nd","rd","th","th","th","th","th","th"][abs(i) % 10])

def get_OSM_data(bbox, KEYTAG='power', type_info=True, round_to=7, print_info=True):
    """ 
    Downloads open street map data within the specified bounding box using the OSM API.
    Only filters items that have the KEYTAG tag. 
    
     INPUT: 
      bbox: tuple of 4 floats (left, bottom, right, top). needed to define rectangular bounding box
      round_to: int. specifies the number of decimal places to round the bounding box to
      print_info: bool. if True, print the number of nodes, ways, and relations found in the bounding box
      type_info: bool. if True, make and return a POWER_TYPES dictionary that keeps track of the number of 
                       occurences for each tag within each power type. 
                       allows later printing with print_osm_info below.
      KEYTAG: str. the tag to filter by. default is 'power' bc we care about the electrical grid here.
    
     OUTPUT: a 2-tuple of DATA dictionary, INDICES dictionary
      DATA is a dictionary of all KEYTAG related items in the bounding box, so that 
      DATA[id] = {'osm_type': 'node', 'lat': lat, 'lon': lon, tags...}         or
                 {'osm_type': 'way', 'nodes': [n1, n2, ...],... tags}     or 
                 {'osm_type': 'relation', 'members': {'nodes': [n1, n2, ...],'ways': [w1, w2, ...]... },... }
      INDICES is a dictionary of all KEYTAG tags with their indices in the dictionary.
        example for all tems with a tag that has 'k'=KEYTAG, 'v'='generator':
          INDICES['generator'] = [id1, id2, ...] 
    """
    # store all power data in dictionary
    DATA = {}
    # keep track of all indices with each power tag 
    # to easily access all power plants, for example
    POWER_IDXS = defaultdict(list)
    # keep track of the type of each power tag. some things might have 
    # multiple types.
    # keep track of the number of occurences for each tag within 
    # each power type
    if type_info:
        POWER_TYPES = defaultdict(dict)

    def update_info_dicts(id, tags):
        ''' update POWER_IDXS and POWER_TYPES '''
        ptype = tags[KEYTAG]
        POWER_IDXS[ptype].append(id)
        # keep track of the number of occurences for each tag within a power type
        if type_info:
            for key, val in tags.items():
                if key not in (KEYTAG, 'lat', 'lon', 'nodes', 'members', 'nnodes'):
                    POWER_TYPES[ptype][key] = POWER_TYPES[ptype].get(key, defaultdict(int))
                    POWER_TYPES[ptype][key][val] += 1
    
    bbox = tuple(map(lambda x: round(x, round_to), bbox))
    Q = queue.Queue()
    Q.put(bbox)
    nbboxes = 1

    nnodes = nways = nrels = 0
    while nbboxes:
        bbox_ = Q.get()
        nbboxes -= 1
        # api request for selected bounding box
        url = "https://api.openstreetmap.org/api/0.6/map?bbox="
        url += ",".join(map(str, bbox_))
        #print(url)
        request = requests.get(url)   # make the api request
        response = request.content    # get the (bytes) response
        try:   # parse the response into a dictionary
            root = ET.fromstring(response)
        except ET.ParseError:
            # we requested too many nodes, so now let's
            # split the bounding box into 4 quadrants
            left, bottom, right, top = bbox_
            mid_x = round((left + right) / 2,round_to)
            mid_y = round((bottom + top) / 2,round_to)
            # add 4 quadrants to the queue
            Q.put((left, bottom, mid_x, mid_y))  # bottom left
            Q.put((mid_x, bottom, right, mid_y)) # bottom right
            Q.put((left, mid_y, mid_x, top))     # top left
            Q.put((mid_x, mid_y, right, top))    # top right
            nbboxes += 4
            continue
        
        # add all KEYTAG related items to dictionary for later use
        for element in root:
            # filter only items with KEYTAG specified
            save = False
            for child in element:
                if child.tag == 'tag':
                    if child.get('k') == KEYTAG:
                        save = True
                        break
            if not save: continue

            # parse element and save to dictionaries
            id = int(element.get('id'))
            if element.tag == 'node':
                tags = {"osm_type": "node", "lat": element.get('lat'), "lon": element.get('lon')}
                for child in element:   # assume all tages are child.tag == 'tag'
                    if child.tag == 'tag':
                        key = child.get('k'); val = child.get('v')
                        if key and val:
                            tags[key] = val
                            if key == KEYTAG: save = True
                        else:
                            print(f"Node {id} has a tag with no key-value: ", child.attrib)
                DATA[id] = tags
                update_info_dicts(id, tags)
                nnodes += 1
            elif element.tag == 'way':
                tags = {'osm_type': "way", 'nodes': []}
                waylen = 0
                for child in element:
                    if child.tag == 'nd':
                        tags['nodes'].append(child.get('ref'))
                        waylen += 1
                    elif child.tag == 'tag':
                        key = child.get('k'); val = child.get('v')
                        if key and val:
                            tags[key] = val
                        else:
                            print(f"Way {id} has a tag with no key-value: ", child.attrib)
                tags['nnodes'] = waylen
                DATA[id] = tags
                update_info_dicts(id, tags)
                nways += 1
            elif element.tag == 'relation':
                tags = {"osm_type": "relation", 'members': {}}
                for member in element:
                    if member.tag == 'member':
                        key = member.get('type')+'s'
                        if key not in tags['members']:
                            tags['members'][key] = []
                        tags['members'][key].append(member.get('ref'))
                    elif member.tag == 'tag':
                        key = child.get('k'); val = child.get('v')
                        if key and val:
                            tags[key] = val
                        else:
                            print(f"Relation {id} has a tag with no key-value: ", child.attrib)
                DATA[id] = tags
                update_info_dicts(id, tags)
                nrels += 1
            else:
                print("Unknown element: ", element)
    if print_info:
        print(f"Bbox {bbox} has {nnodes} nodes, {nways} ways, and {nrels} relations.")
    # print info for each type of KEYTAG
    return DATA, POWER_IDXS, POWER_TYPES


def print_osm_info(POWER_IDXS, POWER_TYPES, KEYTAG='power', indent=4):
    """ prettily prints all openstreetmap information. 
        ensures right-indentation of counts for prettiness via the largest 
        count of a single KEYTAG instance """
    keyinfo = [(key, len(POWER_IDXS[key])) for key in POWER_TYPES.keys()]
    keyinfo.sort(key=lambda x: x[1], reverse=True)
    alen = len(str(max([count for key, count in keyinfo])))
    for x in keyinfo:
        key, nitems = x
        lev0 = indent + alen - len(str(nitems))
        print(' '*lev0+f"{nitems} {KEYTAG}={key} has {len(POWER_TYPES[key])} diff tags:")
        taginfo = [(tag, tagdict, sum(c for c in tagdict.values())) for tag, tagdict in POWER_TYPES[key].items()]
        taginfo.sort(key=lambda x: x[2], reverse=True)
        for tag, tag_counts, num_appearances in taginfo:
            num_different = len(tag_counts)
            names = [(name, count) for name, count in tag_counts.items()]
            names.sort(key=lambda x: x[1], reverse=True)
            lev1 = indent + lev0 + alen - len(str(num_appearances))
            if num_different == 1:
                print(' '*lev1+f"{num_appearances} {tag} = {''.join([k for k in tag_counts.keys()])}")
                continue
            if num_different > 10:  # usually only for the 'name' tag
                print(' '*lev1+f"{num_appearances} {tag} has {num_different} diff. values:")
                print(' '*(lev1+indent)+", ".join([f"({count}) {name}" for name, count in names]))
                continue
            print(' '*lev1+f"{num_appearances} {tag} =")
            for name, count in names:
                # right-align the counts
                clen = len(str(count))
                lev2 = indent + lev1 + alen - clen
                print(' '*lev2+f"{count} {name}")

In [48]:
#bounding box coords around Ft. Lauderdale, FL, power plant
bottom = 26.045381
left = -80.219148
top = 26.087833
right = -80.169414

ft_lauderdale = (left, bottom, right, top)

#print('Getting data for Ft. Lauderdale power plant in Florida')
#Df, If = get_OSM_power_data(ft_lauderdale)

print('Getting data for Shiloh wind farm in California')
shiloh_wind = (-121.9452,38.0780,-121.7295,38.2452)
D, I, T = get_OSM_data(shiloh_wind, print_info=True)


Getting data for Shiloh wind farm in California
Bbox (-121.9452, 38.078, -121.7295, 38.2452) has 4049 nodes, 141 ways, and 8 relations.


The `print_osm_info` function then allows us to print the statistics on how people are labelling the `power` items, so we know which fields may be relevant to keep when doing our power analysis later. 

In [49]:
print_osm_info(I, T)

    1850 power=pole has 1 diff tags:
        1850 osm_type = node
    1649 power=tower has 4 diff tags:
        1649 osm_type = node
          26 design =
                20 flag
                 6 triangle
          26 structure = tubular
           5 size = huge
     529 power=generator has 13 diff tags:
          529 osm_type =
               528 node
                 1 way
          529 generator:method =
               526 wind_turbine
                 2 combustion
                 1 photovoltaic
          529 generator:source =
               526 wind
                 2 gas
                 1 solar
          528 generator:output:electricity =
               173 2 MW
               139 1.8 MW
               111 1.5 MW
                50 2.3 MW
                29 3 MW
                23 0.66 MW
                 2 47 MW
                 1 yes
          453 generator:type =
               450 horizontal_axis
                 2 gas_turbine
                 1 solar_photovoltaic_panel
 

As for `osm_type`, power plants and substations are either closed ways or relations (that effectively function as a closed way). 

Generators can be either a closed way (like a rectangle depicting a solar panel) or a node (like a wind turbine). 

Connections between stations of either line, minor_line, or cable are open ways spanned by towers, portals, pylons, and poles. 

And some transmission lines are connected additionally via a link, transformer, and switch.

We want to convert power data into a `pandapower` network so that we can run subsequent analysis through python. We will follow a similar approach to Johannes Leimhofer when he made OpenGridMap in 2016. 
